<a href="https://colab.research.google.com/github/MrEminent42/wordle-ai/blob/main/WordleAI_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WordleGame

In [2]:
from enum import IntEnum
from termcolor import colored


class WordleGame:
    def __init__(self, answer):
        self.answer = answer.upper()
        self.board = []
        self.is_over = False
        self.win = False

    def __repr__(self):
        s = ""
        # in each line
        for i, line in enumerate(self.board):
            colors = self.get_colors(self.get_line_string(i))
            for i, tile in enumerate(line):
                # for i, char in enumerate(line):
                s += (
                    colored(tile.char, "white")
                    if tile.color == Color.GREY
                    else (
                        colored(tile.char, "magenta")
                        if tile.color == Color.YELLOW
                        else colored(tile.char, "green")
                    )
                )
            s += "\n"

        return s

    def guess(self, guess):
        """Takes a five-letter guess, records this guess on the game's board.
        Returns the array of Colors with each index corresponding to the color of the letter at that index in the guess"""
        tiles = []
        if len(guess) != 5:
            raise ValueError(
                'Wordle guess must be a 5-letter word. Could not guess with word "'
                + guess
                + '".'
            )
        # convert everything to upper case
        guess = guess.upper()
        colors = self.get_colors(guess)
        # log guess to board
        tiles = [Tile(guess[i], colors[i]) for i in range(5)]
        self.board.append(tiles)

        print("LOGGED COLOR " + str(int(colors[1])))

        # check for game over
        if len(self.board) >= 6:
            self.is_over = True
        elif guess == self.answer:
            self.is_over = self.win = True

        # give back list of colors
        return colors

    def get_colors(self, guess):
        """Takes in a five-letter guess, returns an array of Colors with
        each index corresponding to the color of the letter at that index in the guess."""
        if len(guess) != 5:
            raise ValueError(
                'Can only find colors for words of length 5. Could not find colors for word "'
                + guess
                + '"'
            )
        colors = []
        # occurrences_left = {char: self.answer.count(char) for char in self.answer}
        occurrences_left = {}
        # more efficient way of counting num occurences
        for char in self.answer:
            if char in occurrences_left:
                occurrences_left[char] += 1
            else:
                occurrences_left[char] = 1

        for i, char in enumerate(guess):
            # if the character is in the correct place, green
            if self.answer[i] == char:
                colors.append(Color.GREEN)
                occurrences_left[char] -= 1
            # if the character is in the word, but in the wrong place
            elif char in self.answer:
                # if there are stil occurences of this char that have not been accounted for
                if occurrences_left[char] > 0:
                    # append a yellow tile
                    colors.append(Color.YELLOW)
                    # record that we have accounted for this occurence
                    occurrences_left[char] -= 1
                else:
                    colors.append(Color.GREY)
            else:
                colors.append(Color.GREY)

        return colors

    def get_line_string(self, i):
        s = ""
        for tile in self.board[i]:
            s += tile.char
        return s

    def is_over(self):
        return self.is_over

    def run_game(self):
        print("Welcome to Wordle-AI!")
        while not self.is_over:
            self.guess(input("Guess: "))
            print(self)
        if self.win:
            print("Congrats! You found the word in " + str(len(self.board)) + " tries.")
        else:
            print("Darn! You didn't find the word. It was " + self.answer + ".")


class Color(IntEnum):
    GREY = GRAY = 0
    YELLOW = 1
    GREEN = 2


class Tile:
    def __init__(self, character, color):
        self.char = character
        self.color = color

## DQN
Based on https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

### TF Setup

In [4]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,507 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
     |████████████████████████████████| 18.3 MB 1.2 MB/s 
     |████████████████████████████████| 6.4 MB 8.9 MB/s 
     |████████████████████████████████| 462 kB 46.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

### Gym setup

In [ ]:
!pip install "gym>=0.21.0"

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

import random

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

#### Wordle Data Setup

In [7]:
import requests
import io

import urllib.request
import os
from os import path

In [8]:
raw_data_source = "https://raw.githubusercontent.com/tabatkins/wordle-list/main/words"
directory_name = "WordleData"
file_name = "wordList.txt"


def retrieve_data():
    url_request = urllib.request.urlopen(raw_data_source)
    data = url_request.read().decode("utf-8")
    return data


def store_data():
    if os.path.isfile(get_file_path()) == False:
        data = retrieve_data()
        os.mkdir(directory_name)
        file = open(get_file_path(), "w")
        file.write(data)
        file.close


def get_file_path():
    return path.join(directory_name, file_name)


In [54]:
store_data()

find_word = {}
find_num = {}

with open(get_file_path()) as f:
  words = f.readlines()

for i in range(len(words)):
  find_word[i] = words[i]
  find_num[words[i]] = i

num_words = len(find_word)
num_words

12947

### Environment
Based on https://www.tensorflow.org/agents/tutorials/2_environments_tutorial#creating_your_own_python_environment

In [10]:
wordle_rows = 6 # @param {type:"integer"}
wordle_cols = 5  # @param {type:"integer"}
wordle_colors = 3  # @param {type:"integer"}

english_letters = 26  # @param {type:"integer"}

In [11]:
import random

def get_observation_array(game):
  obs = np.zeros((wordle_rows * wordle_cols, english_letters, wordle_colors))
  # for every tile on the board
  for row in range(len(game.board)):
    for col in range(len(game.board[row])):
      char = game.board[row][col].char
      color = game.board[row][col].color
      obs[row * 5 + col,ord(char) - 65,int(color)] = 1

  return obs

def get_reward(game):
  sum = 0
  for row in game.board:
    for tile in row:
      sum += int(tile.color) + 1
  return sum

In [12]:
# test the observation spec
game = WordleGame(find_word[random.randint(0,num_words)])
game.guess("LATER")
print("Answer: ", game.answer)
obs = get_observation_array(game)
print("Tile 0 [L]: ", obs[0][ord("L")-65])
print("Tile 1 [A]: ", obs[1][ord("A")-65])
print("Tile 2 [T]: ", obs[2][ord("T")-65])
print("Tile 3 [E]: ", obs[3][ord("E")-65])
print("Tile 4 [R]: ", obs[4][ord("R")-65])

print("Current reward:", get_reward(game))

LOGGED COLOR 1
Answer:  BIVIA

Tile 0 [L]:  [1. 0. 0.]
Tile 1 [A]:  [0. 1. 0.]
Tile 2 [T]:  [1. 0. 0.]
Tile 3 [E]:  [1. 0. 0.]
Tile 4 [R]:  [1. 0. 0.]
Current reward: 6


In [58]:
class WordleEnvironment(py_environment.PyEnvironment):
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=num_words - 1, name='action'
    )
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(wordle_rows * wordle_cols, english_letters, wordle_colors), dtype=np.int32, minimum=0, maximum=1, name='observation'
    )
    
    self.game = WordleGame(find_word[random.randint(0,num_words)])

  def action_spec(self):
    return self._action_spec
  
  def observation_spec(self):
    return self._observation_spec
  
  def _reset(self):
    self.game = WordleGame(find_word[random.randint(0,num_words)])
    return ts.restart(get_observation_array(self.game), get_reward(self.game)) # TODO update

  def _step(self, action):
    if self.game.is_over: 
      return self.reset()

    # perform the action
    colors = self.game.guess(find_word[action])

    # return the observation state
    # if the game is now over
    if self.game.is_over:
      return ts.termination(get_observation_array(self.game), get_reward(self.game))


    


        

In [14]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

### Training

In [59]:
train_py_env = WordleEnvironment()
print('Observation Spec:')
print(train_py_env.time_step_spec().observation)
print('Reward Spec:')
print(train_py_env.time_step_spec().reward)
print('Action Spec:')
print(train_py_env.action_spec())

Observation Spec:
BoundedArraySpec(shape=(30, 26, 3), dtype=dtype('int32'), name='observation', minimum=0, maximum=1)
Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')
Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=12946)


In [60]:
eval_py_env = WordleEnvironment()
print('Observation Spec:')
print(eval_py_env.time_step_spec().observation)
print('Reward Spec:')
print(eval_py_env.time_step_spec().reward)
print('Action Spec:')
print(eval_py_env.action_spec())

Observation Spec:
BoundedArraySpec(shape=(30, 26, 3), dtype=dtype('int32'), name='observation', minimum=0, maximum=1)
Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')
Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=12946)


In [61]:
# convert to TF Environment
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agent

In [62]:
layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1
num_actions

12947

In [68]:
# helper method for creating a bunch of dense layers
def make_dense_layer(units):
  return tf.keras.layers.Dense(
      units,
      activation="relu",
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'
      )
  )

# create DQN 
dense_layers = [make_dense_layer(units) for units in layer_params]
q_vals_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03
    ),
    bias_initializer=tf.keras.initializers.Constant(-0.2)
)

q_net = sequential.Sequential([tf.keras.layers.Flatten()] + dense_layers + [q_vals_layer])

In [70]:
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer='adam',
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter
)

agent.initialize()